Packages instalation:

In [2]:
pip install eli5

     |████████████████████████████████| 112kB 5.3MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import  mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [11]:
cd "Dataworshop_matrix"

/content/drive/My Drive/Colab Notebooks/Dataworshop_matrix


In [12]:
ls data

men_shoes.csv


Import data:

In [0]:
df = pd.read_csv('data/men_shoes.csv',low_memory=False)

Define run_model() funstion:

In [0]:
def run_model(feats, model= DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['price_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

Feature Engineering:
<br> 
- data normalization

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

- parse features

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [119]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

- Different types of attributes

In [120]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [121]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(
      lambda feats: feats[key] if key in feats else np.nan)

Verify the importance of variables:

In [122]:
keys_stat = {}

for key in tqdm_notebook(keys):
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [123]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_season_cat'] = df['feat_season'].factorize()[0]
df['feat_metal type_cat'] = df['feat_metal type'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]

In [0]:
for key in tqdm_notebook(keys):
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [125]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 1003)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_metal type_cat', 'feat_shape_cat']

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

DecisionTreeRegressor:

In [20]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

RandomForestRegressor:

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=123)
results = run_model(feats, model)

Features importances:

In [134]:
X = df[feats].values
y = df['price_amountmin'].values

m =  RandomForestRegressor(max_depth=5, n_estimators=100, random_state=123)
m.fit(X, y)

print(results)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.21960817427812, 4.21373950229374)


Weight,Feature
0.2530 ± 0.0085,brand_cat
0.0995 ± 0.0015,feat_material_cat
0.0254 ± 0.0022,feat_gender_cat
0.0174 ± 0.0010,feat_brand_cat
0.0110 ± 0.0003,feat_shape_cat
0.0086 ± 0.0008,feat_metal type_cat
0.0051 ± 0.0009,feat_color_cat
0.0044 ± 0.0010,feat_style_cat


In [90]:
df['brand'].value_counts(normalize=True)

nike                   0.097210
puma                   0.033315
ralph lauren           0.028775
vans                   0.021116
new balance            0.020295
                         ...   
coool                  0.000055
cufflink aficionado    0.000055
dkny                   0.000055
nike sb                0.000055
alpha tactical         0.000055
Name: brand, Length: 1732, dtype: float64

In [104]:
df[df['brand'] == 'nike'].features_parsed.sample(10).values

array([{'style': 'shirts & tops', 'condition': 'new with tags'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'velvet brown/dark khaki/gum', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '807242222', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {},
       {'pattern': 'solid print', 'gender': 'men', 'material': 'synthetic', 'brand': 'nike', 'age group': 'adult', 'color': 'black'},
       {'sport': 'boots', 'occasion': 'boots', 'material': 'suede', 'gender': 'men', 'shoe size': '9.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'suede', 'color': 'tawny/hypr orange/brgd bl/rttn', 'model': '654888 284', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'boots', 'manufacturer part number': '654888 284', 'brand': 'nike', 'age group': 'men', 'shoe width': 'd(m)'},
       {'country/region of manufacture': 'china', 'mpn': '684860-003', 'colo

In [0]:
!git add matrix_one/1_day5.ipynb

In [0]:
!git commit -m "Predict Men's Shoes Prices - second part"